In [ ]:
# default_exp expscript

[autoreload of reformer_fastai.reformer failed: Traceback (most recent call last):
  File "/home/morgan/anaconda3/envs/fastai_env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/morgan/anaconda3/envs/fastai_env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/morgan/anaconda3/envs/fastai_env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/morgan/anaconda3/envs/fastai_env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/morgan/anaconda3/envs/fastai_env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/morgan/anaconda3/envs/fastai_env/lib/python3.7/site-packages/

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#exports
from fastcore.all import *
from fastai.basics import *

from reformer_fastai.reformer import LSHLM
from reformer_fastai.data import TwinSequence, MaskTargCallback
from reformer_fastai.metrics import MaskedAccuracy

from reformer_fastai.tracking import *
from reformer_fastai.tracking import WandbCallback

In [ ]:
#all_slow

# Config

Sequence length of 1024 is used in the paper and training for 150 k iterations.

In [ ]:
#export

# Dataloaders
bs=128
sl=1024
train_sz=900
valid_sz=100
n_epochs=1      # we want 150 k iterations according to the paper. Adjust n_epochs according to train_szs and bs

# Model
n_hashes=4
bucket_size=64  # suggested in trax
vocab_sz=128    # specific for the synthetic task
d_model=256
n_layers=1      # specified in paper
n_heads=4
max_seq_len=sl
causal=True
use_lsh=True

# Functions

In [ ]:
#export
def get_dataloaders(bs:int=32, sl:int=64, train_sz:int=500, valid_sz:int=100):
    dls = DataLoaders.from_dsets(TwinSequence(sl, train_sz), TwinSequence(sl, valid_sz), bs=bs, shuffle=False, device='cuda')
    return dls

In [ ]:
#export
def get_model(vocab_sz:int=128, d_model:int=256, n_layers:int=1, n_heads:int=4, 
              max_seq_len:int=64, bucket_size:int=32, n_hashes:int=4, causal:bool=True, use_lsh:bool=True):
    model = LSHLM(vocab_sz=vocab_sz, d_model=d_model, n_layers=n_layers, n_heads=n_heads, max_seq_len=max_seq_len,
              bucket_size=bucket_size, n_hashes=n_hashes, causal=True, use_lsh=True)
    return model

In [ ]:
#export
def get_learner(dls, model):
    learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(ignore_index=-100), 
                metrics=MaskedAccuracy(), cbs=[MaskTargCallback()]).to_fp16()
    return learn

## Checks

# Main

In [ ]:
#export
@call_parse
def run_exp(task:Param(help="Which exeriment task to run", type=str),
         lr:Param(help="Learning rate", type=float, default=1e-3),               
         bs:Param(help="Batch size", type=int, default=bs),
         sl:Param(help="Seqlence length", type=int, default=sl),
         n_epochs:Param(help="Number of epochs", type=int, default=n_epochs),
         train_sz:Param(help="TwinSequence train size", type=int, default=train_sz),
         valid_sz:Param(help="TwinSequence valid size", type=int, default=valid_sz),
         n_hashes:Param(help="Number of LSH Attention hashes", type=int, default=n_hashes),
         bucket_size:Param(help="LSH Attention bucket size", type=int, default=bucket_size),
         vocab_sz:Param(help="Vocab size", type=int, default=vocab_sz),
         d_model:Param(help="Model dimension", type=int, default=d_model),
         n_layers:Param(help="Number of model layers", type=int, default=n_layers),
         n_heads:Param(help="Number of attention heads", type=int, default=n_heads),
         max_seq_len:Param(help="Max sequence length for model embedding", type=int, default=max_seq_len),
         causal:Param(help="Use causal masking", type=int, default=causal),
         use_lsh:Param(help="Use LSH Attention", type=bool, default=use_lsh),
         save_model:Param(help="Save model locally in /models", type=bool, default=True),
         do_wandb_logging:Param(help="Use wandb logging", type=bool, default=False),
         wandb_name:Param(help="wandb run name", type=str, default='my_experiment_name'),
         wandb_group:Param(help="wandb group", type=str, default='TEST'),
         wandb_notes:Param(help="wandb notes", type=str, default='My experiment notes'),
#          wandb_config:Param(help="Use wandb logging", type=bool, default='my_experiment_name'),
         wandb_tags:Param(help="wandb tags", type=list, default=['test']),
         cuda_id:Param(help="Which cuda device to use", type=int, default=0)
                     ):

    """tasks: synthetic, lm, translation"""
                      
    if task == 'synt':
        
        torch.cuda.set_device(cuda_id)

        dls = get_dataloaders(bs=bs, sl=sl, train_sz=train_sz, valid_sz=valid_sz)

        model = get_model(vocab_sz=vocab_sz, d_model=d_model, n_layers=n_layers, n_heads=n_heads, 
                  max_seq_len=max_seq_len, bucket_size=bucket_size, n_hashes=n_hashes, causal=causal, use_lsh=use_lsh)

        learn = get_learner(dls, model)
        
        cbs=[MaskTargCallback()]
        
        if do_wandb_logging:
            try:
                import wandb
                #!wandb login
            except ImportError as e:
                print(e)
                
            # Init wandb
            wandb.init(reinit=True, project="reformer-fastai", entity="fastai_community", 
                   name=wandb_name, group=wandb_group, notes=wandb_notes, tags=wandb_tags, config={})
            
            cbs.append(WandbCallback(log_model=False, log_preds=False))
        print('starting train')
        learn.fit_one_cycle(n_epochs, lr, cbs=cbs)
        
        if save_model:
            import datetime
            now = datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S")
            learn.save(f'{task}_n_hashes-{n_hashes}_use-lsh-{use_lsh}_epohs-{n_epochs}_{now}')
        
    else:
        print('No task run')

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_attention.ipynb.
Converted 03_transformer.ipynb.
Converted 04_reformer.ipynb.
Converted 05_tokenizers.ipynb.
Converted 06_data.ipynb.
Converted 07_metrics.ipynb.
Converted 08_optimizers.ipynb.
Converted 09_tracking.ipynb.
Converted 10_experiment.synthetic-task.ipynb.
Converted 10a_experiment.synthetic-task-comparison.ipynb.
Converted 10b_experiment.synthetic-task-minimal.ipynb.
Converted 11_experiment.enwik8_baseline-Copy1.ipynb.
Converted 11_experiment.enwik8_baseline.ipynb.
Converted 12_experiment.enwik8_sharedQK.ipynb.
Converted 13_experiment.enwik8_reversible.ipynb.
Converted 20_experiment-script.ipynb.
Converted 50_exploration.LSH.ipynb.
Converted index.ipynb.
Converted reproducibility.report_1_reproducibility_summary.ipynb.
